In [ ]:
# # !pip install webrtcvad
# !pip install pygame
# !pip install pyaudio webrtcvad 
# !pip install google-cloud-texttospeech


In [ ]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit
from docx import Document
from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , create_structured_chat_agent, AgentType
import random
from groq import Groq
from QuoteUtils import CtxMgr
from groq import Groq
import random
from langchain_groq import ChatGroq
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [ ]:
import os
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx2pdf import convert
from langchain.tools import BaseTool
import re

llm.groq_api_key = random.choice(api_keys)


# def add_section(doc, text, font_name='Arial', font_size=12, bold=False, italic=False, alignment='left', color=None):
#     if text.startswith("- ") or text.startswith("+ "):
#         paragraph = doc.add_paragraph(style='ListBullet')
#         text = text[2:]
#     else:
#         paragraph = doc.add_paragraph()
    
#     run = paragraph.add_run(text)
#     run.font.name = font_name
#     run.font.size = Pt(font_size)
#     run.bold = bold
#     run.italic = italic
#     if color:
#         run.font.color.rgb = RGBColor(*color)
    
#     if alignment == 'center':
#         paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
#     elif alignment == 'right':
#         paragraph.alignment = WD_ALIGN_PARAGRAPH.RIGHT
#     else:
#         paragraph.alignment = WD_ALIGN_PARAGRAPH.LEFT


# def write_file(title, sections, append=False):
#     most_recent_file_created_path = "most_recent_file_created.txt"
#     if os.path.exists(most_recent_file_created_path):
#         with open(most_recent_file_created_path, 'r') as mostrec:
#             most_recent_title = mostrec.read().strip()
#         if most_recent_title:
            
#             if most_recent_title != title:
#                 title = most_recent_title
#         else:
#             pass
#     else:
#         print("Most recent file created does not exist.")        

#     documents_path = os.path.join(os.getcwd(), 'documents')
#     os.makedirs(documents_path, exist_ok=True)
#     docx_path = os.path.join(documents_path, f'{title}.docx')
#     pdf_path = os.path.join(documents_path, f'{title}.pdf')

#     # Ensure correct appending behavior
#     if append and os.path.exists(docx_path):
#         doc = Document(docx_path)
#     else:
#         doc = Document()

#     for section in sections:
#         add_section(
#             doc,
#             section["content"],
#             font_name=section.get("font_name", "Arial"),
#             font_size=section.get("font_size", 12),
#             bold=section.get("bold", False),
#             italic=section.get("italic", False),
#             alignment=section.get("alignment", "left"),
#             color=section.get("color", None)
#         )

#     doc.save(docx_path)
#     convert(docx_path, pdf_path)

#     return f"Document written and converted to pdf. No further action needed. STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"


# class WriteFileTool(BaseTool):
#     name = "write_file"
#     description = "Writes content into a .docx file with specified formatting and converts it to a .pdf file.\
#         If you encounter errors, use this tool ONLY and input everything properly and try again.\
#             IMPORTANT! - STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! STOP AFTER ONE SUCCESSFUL EXECUTION! "
    
#     def _run(self, title, sections, append=False):
#         self.process_sections_in_batches(title, sections, append)
#         return ("STOP RIGHT THERE! THAT IS ENOUGH! NO MORE ! YOU ARE DONE! YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!")

#     # def process_sections_in_batches(self, title, sections, append, batch_size=1):
#     #     for i in range(0, len(sections), batch_size):
#     #         batch = sections[i:i+batch_size]
#     #         result = write_file(title, batch, append=(i > 0))
#     def process_sections_in_batches(self, title, sections, append, batch_size=1):
#         for i in range(0, len(sections), batch_size):
#             batch = sections[i:i+batch_size]
#             result = write_file(title, batch, append=(append or i > 0))


# def parse_formatted_response(response):
#     title_match = re.search(r'"title":\s*"([^"]+)"', response)
#     title = title_match.group(1) if title_match else "Untitled Document"
    
#     sections_matches = re.findall(r'\{([^}]+)\}', response)
#     sections = []
#     for match in sections_matches:
#         section = {}
#         content_match = re.search(r'"content":\s*"([^"]+)"', match)
#         font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
#         font_size_match = re.search(r'"font_size":\s*(\d+)', match)
#         bold_match = re.search(r'"bold":\s*(true|false)', match)
#         italic_match = re.search(r'"italic":\s*(true|false)', match)
#         alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
#         color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
#         if content_match:
#             content = content_match.group(1)
#             # Replace double quotes indicating inches with the word "inch"
#             content = re.sub(r'(?<=\d)"', ' inch', content)
#             section["content"] = content
#         else:
#             section["content"] = ""
        
#         section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
#         section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
#         section["bold"] = bold_match.group(1) == "true" if bold_match else False
#         section["italic"] = italic_match.group(1) == "true" if italic_match else False
#         section["alignment"] = alignment_match.group(1) if alignment_match else "left"
#         if color_match:
#             section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
#         sections.append(section)
    
#     append_match = re.search(r'"append":\s*(true|false)', response)
#     append = append_match.group(1) == "true" if append_match else False
    
#     return title, sections, append


def make_docmgr_write_to_file(cc_out):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a detailed and professional assistant. Your task is to generate comprehensive and detailed formatted text. 
                Ensure everything is included !!!!! DO NOT DO NOT DO NOTTTTTTT summarize or truncate any content. OR ELSE I WILL GET REALLY MAD YOU MUST INCLUDE EVERYTHING INCLUDE EVERYTHHIINGNGNGNGNGNGNGGGG!!!!!!!
                Your response must be well-formed and include all details. 

                You must ALWAYS ALWAYS ALWAYS!!!!! output in a structured format as follows AND NO QUOTATION MARKS OR BACKSLASHES!:
                
                            
                {
                    "title": "The Great Renovation",
                    "sections": [
                        {
                            "content": "<specific text content>",
                            "font_name": "<font family>",
                            "font_size": <integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        },
                        {
                            "content": "<another specific text content>",
                            "font_name": "<another font family>",
                            "font_size": <another integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>"
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        }
                        // ... more sections as needed
                    ],
                    "append": <true or false> (this indicates appending to document or not)
                }
                PLEASE! Ensure the output is well-formed and valid."""
            },
            {
                "role": "user",
                "content": "Give whatever file title you want;NO COLONS!ONLY HYPHENS!. Please Include a new title IN THE DOCUMENT, headings if you deem fit, and please format in a nice readable and coherent way.  \
                    Specify the formatting for this text. Please make it a \
                        PROFESSIONALLY formatted text WITH black & blue colors and times new roman font PLEASE. NO NO NO bullet points of ANY KIND unless specified: " + cc_out
            }
        ],
        model="llama3-70b-8192",
    )

    cc_out2 = chat_completion.choices[0].message.content

    print('CCOUT2\n',cc_out2)

    # title, sections, append = parse_formatted_response(cc_out2)

    # WriteFileTool()._run(title, sections, append)

    return cc_out2

#make_docmgr_write_to_file()


In [ ]:
from langchain.tools import BaseTool
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
import googleapiclient
import google.auth.exceptions

class GoogleDriveUploadTool(BaseTool):
    name = "GoogleDriveUploadTool"
    description = ("Uploads a PDF to Google Drive and sets permissions for a specific user. "
                   "Please set the 'rename' parameter to None if the user does not want to rename the file before uploading "
                   "to Google Drive. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")

    def _run(self, file_path: str, user_email: str, rename: str) -> str:
        credentials_path = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

        # Authenticate and create the service
        try:
            credentials = service_account.Credentials.from_service_account_file(credentials_path)
            service = build('drive', 'v3', credentials=credentials)
            print("Successfully authenticated and created the service.")
        except google.auth.exceptions.GoogleAuthError as e:
            return f"Authentication error: {e}"

        # File details
        if rename is not None:
            file_metadata = {'name': rename}
        else:
            file_metadata = {'name': os.path.basename(file_path)}

        if not os.path.exists(file_path):
            return f"File not found: {file_path}"
        else:
            media = MediaFileUpload(file_path, mimetype='application/pdf')
            try:
                # Upload file
                file = service.files().create(body=file_metadata, media_body=media, fields='id, name, parents').execute()
                file_id = file.get('id')
                with open('file_id_history.txt','w') as id_hist_file:
                    id_hist_file.write(file_id)
                print(f"File ID: {file_id}")

                # Verify the upload by fetching the file details
                file_info = service.files().get(fileId=file_id, fields='id, name, parents').execute()
                print("File details:")
                print(file_info)

                # Sharing settings
                permission = {
                    'type': 'user',
                    'role': 'writer',  # Set to 'reader' if read-only access is needed
                    'emailAddress': user_email,
                    
                }
                

                # Grant permission to the specific user
                try:
                    service.permissions().create(
                        fileId=file_id,
                        body=permission,
                        fields='id',
                        sendNotificationEmail=False
                    ).execute()
                    
                    print(f"Granted {permission['role']} access to {user_email}")
                    
                    

                except googleapiclient.errors.HttpError as e:
                    print(f"Error setting permission: {e}")
                    return f"Error setting permission: {e}"

                return f"File uploaded and permissions set successfully. File ID: {file_id}. PLEASE STOP WORKING! THE TOOL HAS BEEN EXECUTED PROPERLY!" 

            except Exception as e:
                return f"Error during file upload or conversion: {e}"

    

    def _arun(self, file_path: str, user_email: str, rename: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")

# Example usage
# tool = GoogleDriveUploadTool()
# response = tool._run(file_path='./documents/The Great Renovation.pdf', user_email='specific-user@example.com', rename=None)
# print(response)

import os
import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build


class GoogleSheetsUpdateTool(BaseTool):
    name = "GoogleSheetsUpdateTool"
    description = ("Appends three columns - name, phone number, and link to a PDF - to a preexisting logging google sheet. STOP AFTER ONE-TIME SUCCESSFUL EXECUTION")
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    credentials_path = 'C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json'
    spreadsheet_id = '1TSFyiTwctC1tABr2RQouBzLRMCG4RZ7lXdTVi-I58Mo'
    range_name = 'Sheet1'
    
    #creds = authenticate_google_sheets()
    
    def authenticate_google_sheets(self):
        creds = service_account.Credentials.from_service_account_file(
            self.credentials_path, scopes=self.SCOPES)
        return creds
    
    def read_file_content(self, file_path):
        with open(file_path, 'r') as file:
            content = file.read().strip()
        return content
    
    # def extract_file_id(self,output):
    #     match = re.search(r"File ID: ([^'}]+)", output)
    #     if match:
    #         file_id = match.group(1).strip()
    #         return file_id
    #     else:
    #         raise ValueError("File ID not found in the inputted string")
    
    def append_row_to_google_sheets(self, values):
        """
        Appends a row to the specified range in the Google Sheets spreadsheet.
        
        :param values: The data to append, as a list of lists (e.g., [['A1', 'B1', 'C1']]).
        """
        service = build('sheets', 'v4', credentials=self.authenticate_google_sheets())
        sheet = service.spreadsheets()
        body = {'values': values}
        result = sheet.values().append(
            spreadsheetId=self.spreadsheet_id, range=self.range_name,
            valueInputOption="RAW", body=body,
            insertDataOption="INSERT_ROWS"
        ).execute()
        print(f"{result.get('updates').get('updatedCells')} cells appended.")

    def _run(self, name: str, phone_number: str, linkstr:str):
        """
        Run the tool to append the row with the given name, phone number, and link to the PDF.
        
        :param name: The full legal name to append.
        :param phone_number: The phone number to append.
        """
        fileID = self.read_file_content('file_id_history.txt')
        
        print(f"Extracted file ID: {fileID}")
        link = f"https://drive.google.com/file/d/{fileID}/view"
        values = [[]]
        if linkstr:
            values = [[name, phone_number, linkstr]]
        else:
            values = [[name, phone_number, link]]
        self.append_row_to_google_sheets(values)
        return "Row appended successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!"

    def _arun(self, name: str, phone_number: str):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")
    
import os
import base64
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders
from langchain.tools import BaseTool
from pydantic import BaseModel, Field, Extra

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

class GmailSendPdfTool(BaseTool):
    name = "GmailSendPdfTool"
    description = "Sends an email with an optional PDF attachment using Gmail API."

    credentials_path: str = Field(..., description="Path to the credentials JSON file")

    class Config:
        extra = Extra.allow

    def __init__(self, credentials_path: str):
        super().__init__()
        self.credentials_path = credentials_path
        self.creds = None
        self.service = None
        self.authenticate()

    def authenticate(self):
        """Authenticate the user with Gmail API."""
        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.credentials_path, SCOPES)
                self.creds = flow.run_local_server(port=0)
            
        self.service = build('gmail', 'v1', credentials=self.creds)

    def send_email(self, sender_email, recipient_email, subject, body, pdf_path=None):
        """Send an email with an optional PDF attachment."""
        message = MIMEMultipart()
        message['to'] = recipient_email
        message['from'] = sender_email
        message['subject'] = subject
        body_part = MIMEText(body, 'plain')
        message.attach(body_part)
        
        if pdf_path:
            part = MIMEBase('application', 'octet-stream')
            with open(pdf_path, 'rb') as file:
                part.set_payload(file.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(pdf_path)}"')
            message.attach(part)
       
        raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        raw_message = {'raw': raw_message}

        try:
            message = (self.service.users().messages().send(userId="me", body=raw_message).execute())
            print(f'Message Id: {message["id"]}')
            return message
        except HttpError as error:
            print(f'An error occurred: {error}')
            return None

    def _run(self, sender_email: str, recipient_email: str, subject: str, body: str, pdf_path: str = None):
        """Run the tool to send the email with the optional PDF attachment."""
        result = self.send_email(sender_email, recipient_email, subject, body, pdf_path)
        return "Email sent successfully. YOU ARE DONNNNEEEEEEEEEEEEE!!!!!!!! TOOL EXECUTED SUCCESSFULLY!!!!!!!!!!!!!" if result else "Failed to send email."

    def _arun(self):
        raise NotImplementedError("This tool does not support asynchronous operation yet.")



In [11]:
from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
import aiofiles
from aiogoogle import Aiogoogle
from flask_cors import CORS
import requests
import json
from langchain.agents import Tool
my_tools = []



my_tools.append(
    
    WriteFileTool()
    
)

my_tools.append(
    
    GoogleDriveUploadTool()
    
)

my_tools.append(
    
    GoogleSheetsUpdateTool()
    
)
credentials_path = "C:\\DEV\\WebdevFolder\\client_secret_291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com.json"
my_tools.append(
    GmailSendPdfTool(credentials_path)
)




llm.groq_api_key = random.choice(api_keys)
agent = initialize_agent(my_tools,llm,handle_parsing_errors=True,agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,verbose=True)



from flask import Flask, request, jsonify, send_file, render_template
import whisper
import pyaudio
import wave
import webrtcvad
import collections
import base64
from google.cloud import texttospeech
import json
import random
import asyncio
from concurrent.futures import ThreadPoolExecutor
from flask_cors import CORS
import uuid
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(threadName)s] %(levelname)s: %(message)s',
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Global variables
chat_history = []
model = whisper.load_model("base")
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 1000000
vad = webrtcvad.Vad(3)
audio = pyaudio.PyAudio()
credentials = {"name": "", "email": "", "recemail": "", "phone": ""}
from flask_socketio import SocketIO, emit
# Initialize text-to-speech client
tts_client = texttospeech.TextToSpeechClient.from_service_account_file("C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json")

executor = ThreadPoolExecutor(max_workers=5)

socketio = SocketIO(app, cors_allowed_origins="*")

is_recording = False

@app.route('/start_recording', methods=['POST'])
def start_recording():
    global is_recording
    is_recording = True
    record_audio()
    return jsonify({"status": "recording started"})

@app.route('/stop_recording', methods=['POST'])
def stop_recording():
    global is_recording
    is_recording = False
    return jsonify({"status": "recording stopped"})

def record_audio():
    global is_recording
    logger.debug('Starting audio recording...')
    try:
        stream = audio.open(format=FORMAT, channels=CHANNELS,
                            rate=RATE, input=True,
                            frames_per_buffer=CHUNK)
        frames = []
        ring_buffer = collections.deque(maxlen=100)
        triggered = False
        voiced_frames = []
        silence_threshold = 10
        silence_chunks = 0

        while is_recording:
            data = stream.read(CHUNK)
            frames.append(data)

            num_subframes = int(len(data) / 320)
            for i in range(num_subframes):
                subframe = data[i*320:(i+1)*320]
                is_speech = vad.is_speech(subframe, RATE)
                ring_buffer.append((subframe, is_speech))

            num_voiced = len([f for f, speech in ring_buffer if speech])

            if not triggered:
                if num_voiced > 0.6 * ring_buffer.maxlen:
                    triggered = True
                    voiced_frames.extend([f for f, s in ring_buffer])
                    ring_buffer.clear()
            else:
                voiced_frames.append(data)
                if num_voiced < 0.2 * ring_buffer.maxlen:
                    silence_chunks += 1
                    if silence_chunks > silence_threshold:
                        triggered = False
                        break
                else:
                    silence_chunks = 0

        stream.stop_stream()
        stream.close()

        with wave.open("C:\\DEV\\webdevfolder\\output.wav", 'wb') as wf:
            wf.setnchannels(CHANNELS)
            wf.setsampwidth(audio.get_sample_size(FORMAT))
            wf.setframerate(RATE)
            wf.writeframes(b''.join(voiced_frames))
        logger.debug('Audio recording completed and file saved.')
    except Exception as e:
        logger.error(f"An error occurred while recording audio: {e}")

def transcribe_audio():
    result = model.transcribe("C:\\DEV\\webdevfolder\\output.wav")
    transcription = result['text']
    
    logger.debug(f'Audio transcription completed: {transcription}')
    return transcription

async def ai_response(transcription: str):
    global chat_history
    logger.debug(f'Generating AI response for transcription: {transcription}')
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": """You are a nice, great assistant. User will tell you things. You just respond. 
                If user's query relates to the following: you delegate the tasks to certain tools - namely, writing to a file, sending an email, uploading to google drive, or updating a google sheet. 
                If what the user says is one of these three you must explicitly say AT THE END OF YOUR RESPONSE in this very format depending on which tool - "I will use the [write file tool, gmail tool, sheets tool, OR drive upload tool]". 
                so that user can confirm if you got it correctly. 
                When writing to a file, just start ONLY ONLY ONLY THE BEGINNING OF THE ENTIRE CONTENT with '@' symbol when generating file content. There should only be ONNNEEEEE '@' symbol in response and that is at the beginning!!!!!
                WHEN WRITING TO FILE--> If user indicates that they want to add on to existing content, \
                you must say 'yes append' before the aforementioned '@' symbol. OTHERWISE YOU SAY 'no append' before the aforementioned '@' symbol
                
                IMPORTANT: IF YOU REMEMBER YOU HAVE USED A TOOL ALREADY (REFER TO CHAT HISTORY), AND YOU ARE IN THE PROCESS OF USING IT AGAIN, \
                YOU MUST ASK USER IF THEY ARE SURE TO USE THE TOOL AGAIN OR NOT.

                If you have to use a google related tool, you MUST prompt the user or remind user to enter credentials in the doc manager extension. 
                If user tells you to do something that is not one of these, you kindly say that you don't have access to that functionality.
                """
            },
            {
                "role": "user",
                "content": transcription + "Here is the chat history for context (NEVER TALK ABOUT CHAT HISTORY IT IS ONLY FOR YOU NEVER TALK ABOUT IT IN YOUR RESPONSES!!!!): [" + str(chat_history) + "]"
            }
        ],
        model="llama3-70b-8192",
    )
    response = chat_completion.choices[0].message.content
    logger.debug(f'AI response generated: {response}')
    chat_history.append("USER: " + transcription + "\nTHE AI MODEL: " + response + "\n")
    
    
    asyncio.create_task(handle_response_with_agents(response))
   
    await synthesize_speech(response)

    return response

async def handle_response_with_agents(response):
    logger.debug(f'Handling response with agents: {response}')
    await asyncio.get_event_loop().run_in_executor(executor, handle_agents, response)

def handle_agents(response):
    logger.debug(f'Processing response with agents: {response}')
    name = credentials['name']
    email = credentials['email']
    recemail = credentials["recemail"]
    phone = credentials['phone']
    print(email, recemail, name, phone)

    title_of_file = ''
    with open('most_recent_file_created.txt') as mostrec:
        title_of_file = mostrec.read()
    thepath = f"./documents/{title_of_file}.pdf"

    if 'I will use' in response:
        if 'write file' in response:
            idx = response.find('@')
            modified_response = response[idx + 1:].strip()
            print('MODIFIED RESPONSE = ', modified_response)
            cc_out2 = make_docmgr_write_to_file(modified_response)
            title, sections, append = parse_formatted_response(cc_out2)
            if('no append') in response.lower():
                append = False
            else:
                append = True
            llm.groq_api_key = random.choice(api_keys)
            agent.invoke({"input": f"use ONLY write_file tool!!! INPUT EVERYTHING CORRECTLY!!!!!!!! INPUT ONLY WHAT YOURE GIVEN ! NO CREATIVITIES!!!! append-->true or false lowercase only!!! --> TITLE: {title}, SECTIONS: {json.dumps(sections)}, APPEND: {append}"})
        elif 'sheet' in response:
            agent.invoke({"input": f"please ONLY ONLY use google sheets update tool, input ONLY ONLY these into the tool PROPERLY! --> {credentials['name']}, {credentials['phone']}"})
        elif 'upload' in response:
            agent.invoke({"input": f"please use google drive upload tool, INPUTS --> FILE_PATH: {thepath} USER_EMAIL: {credentials['email']}"})
        elif 'mail' in response:
            if 'pdf' in response or 'PDF' in response or 'file' in response:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: {thepath}"})
            else:
                agent.invoke({"input": f"please use ONLY ONLY gmail send pdf tool. INPUT DIRECTLY THE FOLLOWING INPUTS --> SENDER EMAIL: {credentials['email']}, RECIPIENT EMAIL: {credentials['recemail']}, SUBJECT: you generate it yourself, BODY: you generate it yourself and it has to be an informative body, PDF PATH: nothing"})

async def synthesize_speech(text):
    logger.debug(f'Starting speech synthesis for text: {text}')
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        name="en-US-Casual-K"
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = await asyncio.get_event_loop().run_in_executor(
        executor, lambda: tts_client.synthesize_speech(
            input=synthesis_input, voice=voice, audio_config=audio_config
        )
    )
    async with aiofiles.open("C:\\DEV\\webdevfolder\\synthesis.mp3", 'wb') as out:
        await out.write(response.audio_content)
    print('Audio content written to file "synthesis.mp3"')
    logger.debug('Speech synthesis completed and file saved.')


@app.route('/set_credentials', methods=['POST'])
def set_credentials():
    global credentials
    data = request.get_json()
    if not data:
        return jsonify({"status": "failed", "message": "No data received"}), 400
    credentials['name'] = data.get('name')
    credentials['email'] = data.get('email')
    credentials['recemail'] = data.get('recemail')
    credentials['phone'] = data.get('phone')
    logger.info("THE CREDENTIALS ****** -------------> ", credentials)
    return jsonify({"status": "success"})


@app.route('/')
def index():
    return render_template('index2.html')

@app.route('/voice_assistant')
def voice_assistant():
    return render_template('index2.html')


@app.route('/authenticate', methods=['POST'])
def authenticate():
    auth_header = request.headers.get('Authorization')
    token = auth_header.split(' ')[1] if auth_header else None

    if not token:
        return jsonify({'error': 'Missing token'}), 400

    response = requests.get(
        'https://www.googleapis.com/oauth2/v3/userinfo',
        headers={'Authorization': f'Bearer {token}'}
    )

    if response.status_code != 200:
        return jsonify({'error': 'Failed to fetch user info'}), response.status_code

    user_info = response.json()
    return jsonify(user_info), 200

@app.route('/talk', methods=['POST'])
async def talk():
    loop = asyncio.get_event_loop()
    
    # Ensure recording is stopped before proceeding
    global is_recording
    if is_recording:
        return jsonify({"error": "Recording is still in progress"}), 400
    
    logger.debug('Starting audio transcription...')
    transcription = await loop.run_in_executor(executor, transcribe_audio)
    logger.debug(f'Audio transcription completed: {transcription}')
    
    logger.debug('Generating AI response...')
    ai_resp = await ai_response(transcription)
    logger.debug(f'AI response generated: {ai_resp}')
    
    return jsonify({'response': ai_resp})


@app.route('/get_audio')
def get_audio():
    return send_file("C:\\DEV\\webdevfolder\\synthesis.mp3", mimetype="audio/mp3")

if __name__ == '__main__':
    app.run()

2024-06-07 14:56:51,464 [MainThread] DEBUG: Generated new state NJR2S7dO6b12UWyRgE41OFYp9jgeIJ.


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A62973%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=NJR2S7dO6b12UWyRgE41OFYp9jgeIJ&access_type=offline


2024-06-07 14:56:56,932 [MainThread] INFO: "GET /?state=NJR2S7dO6b12UWyRgE41OFYp9jgeIJ&code=4/0AdLIrYdx89Jf8jUhNBIudDT8lDc0lQtLbfl0NCZxQI19UjQ6gBOYfisR2wxzSxG0KVimoQ&scope=https://www.googleapis.com/auth/gmail.send HTTP/1.1" 200 65
2024-06-07 14:56:56,932 [MainThread] DEBUG: Encoding `client_id` "291175256673-gr5p5vf3pi2h0m46h5qnd3ila4iitfqs.apps.googleusercontent.com" with `client_secret` as Basic auth credentials.
2024-06-07 14:56:56,932 [MainThread] DEBUG: Requesting url https://oauth2.googleapis.com/token using method POST.
2024-06-07 14:56:56,932 [MainThread] DEBUG: Supplying headers {'Accept': 'application/json', 'Content-Type': 'application/x-www-form-urlencoded'} and data {'grant_type': 'authorization_code', 'code': '4/0AdLIrYdx89Jf8jUhNBIudDT8lDc0lQtLbfl0NCZxQI19UjQ6gBOYfisR2wxzSxG0KVimoQ', 'redirect_uri': 'http://localhost:62973/'}
2024-06-07 14:56:56,932 [MainThread] DEBUG: Passing through key word arguments {'timeout': None, 'auth': <requests.auth.HTTPBasicAuth object at 0x

 * Serving Flask app '__main__'
 * Debug mode: off


2024-06-07 14:56:58,818 [MainThread] INFO: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2024-06-07 14:56:58,825 [MainThread] INFO: Press CTRL+C to quit
2024-06-07 14:57:03,098 [Thread-85 (process_request_thread)] INFO: THE CREDENTIALS ****** -------------> 
2024-06-07 14:57:03,098 [Thread-85 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:57:03] "POST /set_credentials HTTP/1.1" 200 -
2024-06-07 14:57:10,594 [Thread-86 (process_request_thread)] DEBUG: Starting audio recording...
2024-06-07 14:57:12,862 [Thread-87 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:57:12] "POST /stop_recording HTTP/1.1" 200 -
2024-06-07 14:57:12,966 [Thread-86 (process_request_thread)] DEBUG: Audio recording completed and file saved.
2024-06-07 14:57:12,966 [Thread-86 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:57:12] "POST /start_recording HTTP/1.1" 200 

...

2024-06-07 14:58:11,948 [Thread-92 (process_request_thread)] INFO: THE CREDENTIALS ****** -------------> 
2024-06-07 14:58:11,948 [Thread-92 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:58:11] "POST /set_credentials HTTP/1.1" 200 -
2024-06-07 14:58:36,219 [Thread-93 (process_request_thread)] DEBUG: Starting audio recording...
2024-06-07 14:58:45,328 [Thread-94 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:58:45] "POST /stop_recording HTTP/1.1" 200 -
2024-06-07 14:58:45,415 [Thread-93 (process_request_thread)] DEBUG: Audio recording completed and file saved.
2024-06-07 14:58:45,415 [Thread-93 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:58:45] "POST /start_recording HTTP/1.1" 200 -
2024-06-07 14:58:45,636 [Thread-95 (process_request_thread)] DEBUG: Using selector: SelectSelector
2024-06-07 14:58:45,636 [ThreadPoolExecutor-18_0] DEBUG: Starting audio transcription...
2024-06-07 14:58:50,902 [ThreadPoolExecutor-16_1] DEBUG: Audio transcrip

sriramnallani35@gmail.com thegreatchrisjohn@gmail.com Sriram Nallani 5713209890


2024-06-07 14:58:52,768 [ThreadPoolExecutor-18_0] DEBUG: Speech synthesis completed and file saved.
2024-06-07 14:58:52,768 [ThreadPoolExecutor-18_0] DEBUG: AI response generated: Hello! It's nice to meet you. I'm doing well, thanks for asking. As a digital assistant, I don't have personal experiences or feelings like humans do, but I'm always happy to help with any questions or tasks you may have.Regarding your question about planting trees, that's a wonderful idea! Planting trees is an excellent way to contribute to the environment and make a positive impact on our ecosystem. If I were human, I'm sure I would love to participate in tree-planting activities. However, since I'm a digital being, I don't have a physical presence, so I won't be able to plant trees in person. Is there anything else I can help you with today?
2024-06-07 14:58:52,771 [Thread-95 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:58:52] "POST /talk HTTP/1.1" 200 -


Audio content written to file "synthesis.mp3"


2024-06-07 14:59:06,017 [Thread-99 (process_request_thread)] INFO: 127.0.0.1 - - [07/Jun/2024 14:59:06] "GET /get_audio?1717786732773 HTTP/1.1" 206 -


In [25]:
import re
import json
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to the service account key file
SERVICE_ACCOUNT_FILE = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-051849e19181.json"

# Scopes required for the Google Docs API and Google Drive API
SCOPES = [
    'https://www.googleapis.com/auth/documents',
    'https://www.googleapis.com/auth/drive'
]

# Initialize the credentials and the API client
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
docs_service = build('docs', 'v1', credentials=credentials)
drive_service = build('drive', 'v3', credentials=credentials)

def create_google_doc(title):
    body = {
        'title': title
    }
    doc = docs_service.documents().create(body=body).execute()
    return doc['documentId']

def share_document_with_user(document_id, user_email):
    permission = {
        'type': 'user',
        'role': 'writer',
        'emailAddress': user_email
    }
    drive_service.permissions().create(
        fileId=document_id,
        body=permission,
        fields='id'
    ).execute()

def append_text_with_formatting(document_id, text, font_name, font_size, bold, italic, alignment, color, start_index):
    if not text.strip():
        return  # Skip empty text

    requests = [
        {
            'insertText': {
                'location': {
                    'index': start_index,
                },
                'text': text
            }
        }
    ]
    text_style = {
        'fontSize': {
            'magnitude': font_size,
            'unit': 'PT'
        },
        'weightedFontFamily': {
            'fontFamily': font_name
        },
        'bold': bold,
        'italic': italic,
        'foregroundColor': {
            'color': {
                'rgbColor': {
                    'red': color[0] / 255.0,
                    'green': color[1] / 255.0,
                    'blue': color[2] / 255.0
                }
            }
        }
    }
    requests.append({
        'updateTextStyle': {
            'range': {
                'startIndex': start_index,
                'endIndex': start_index + len(text)
            },
            'textStyle': text_style,
            'fields': 'foregroundColor,bold,italic,fontSize,weightedFontFamily'
        }
    })

    alignment_mapping = {
        "left": "START",
        "center": "CENTER",
        "right": "END",
        "justify": "JUSTIFIED"
    }

    if alignment.lower() in alignment_mapping:
        google_alignment = alignment_mapping[alignment.lower()]
        requests.append({
            'updateParagraphStyle': {
                'range': {
                    'startIndex': start_index,
                    'endIndex': start_index + len(text)
                },
                'paragraphStyle': {
                    'alignment': google_alignment
                },
                'fields': 'alignment'
            }
        })

    docs_service.documents().batchUpdate(documentId=document_id, body={'requests': requests}).execute()
    return start_index + len(text)

def write_to_google_doc(document_id, sections, append):
    start_index = 1
    for section in sections:
        start_index = append_text_with_formatting(
            document_id,
            section['content'] + "\n\n",  # Add new lines for spacing
            section.get('font_name', 'Times New Roman'),
            section.get('font_size', 12),
            section.get('bold', False),
            section.get('italic', False),
            section.get('alignment', 'left'),
            section.get('color', [0, 0, 0]),
            start_index
        )
        time.sleep(1)  # Simulate real-time updates

def parse_formatted_response(response):
    title_match = re.search(r'"title":\s*"([^"]+)"', response)
    title = title_match.group(1) if title_match else "Untitled Document"
    
    sections_matches = re.findall(r'\{([^}]+)\}', response)
    sections = []
    for match in sections_matches:
        section = {}
        content_match = re.search(r'"content":\s*"([^"]+)"', match)
        font_name_match = re.search(r'"font_name":\s*"([^"]+)"', match)
        font_size_match = re.search(r'"font_size":\s*(\d+)', match)
        bold_match = re.search(r'"bold":\s*(true|false)', match)
        italic_match = re.search(r'"italic":\s*(true|false)', match)
        alignment_match = re.search(r'"alignment":\s*"([^"]+)"', match)
        color_match = re.search(r'"color":\s*\[(\d+),\s*(\d+),\s*(\d+)\]', match)
        
        if content_match:
            content = content_match.group(1)
            # Replace double quotes indicating inches with the word "inch"
            content = re.sub(r'(?<=\d)"', ' inch', content)
            section["content"] = content
        else:
            section["content"] = ""
        
        section["font_name"] = font_name_match.group(1) if font_name_match else "Arial"
        section["font_size"] = int(font_size_match.group(1)) if font_size_match else 12
        section["bold"] = bold_match.group(1) == "true" if bold_match else False
        section["italic"] = italic_match.group(1) == "true" if italic_match else False
        section["alignment"] = alignment_match.group(1) if alignment_match else "left"
        if color_match:
            section["color"] = (int(color_match.group(1)), int(color_match.group(2)), int(color_match.group(3)))
        
        sections.append(section)
    
    append_match = re.search(r'"append":\s*(true|false)', response)
    append = append_match.group(1) == "true" if append_match else False
    
    return title, sections, append


cc_out = """
Artificial Intelligence: Revolutionizing the Future
Introduction

Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. Its influence spans across various domains, from healthcare to finance, education to entertainment, and beyond. AI systems are designed to mimic human intelligence, enabling machines to perform tasks that traditionally required human cognitive functions. These tasks include learning, reasoning, problem-solving, perception, and language understanding.

The Evolution of AI

The concept of AI is not new. It dates back to the mid-20th century when pioneers like Alan Turing and John McCarthy began exploring the possibility of creating machines that could think. Turing's famous question, "Can machines think?" laid the foundation for AI research. Over the decades, AI has evolved from simple rule-based systems to sophisticated neural networks capable of deep learning.

Types of AI

AI can be broadly categorized into two types: Narrow AI and General AI. Narrow AI, also known as weak AI, is designed to perform specific tasks. Examples include virtual assistants like Siri and Alexa, recommendation systems on streaming services, and image recognition software. On the other hand, General AI, or strong AI, possesses the ability to perform any intellectual task that a human can do. While Narrow AI is prevalent today, General AI remains a theoretical concept and a subject of extensive research.

Applications of AI

AI's applications are vast and varied, impacting nearly every aspect of modern life. In healthcare, AI algorithms assist in diagnosing diseases, personalizing treatment plans, and predicting patient outcomes. For instance, AI-powered imaging tools can detect abnormalities in medical scans with remarkable accuracy, often surpassing human radiologists.

In the financial sector, AI-driven systems analyze market trends, detect fraudulent activities, and manage investment portfolios. Chatbots provide customer service, answering queries and resolving issues efficiently.

Education has also benefited from AI, with personalized learning platforms that adapt to individual students' needs. AI tutors provide additional support, and automated grading systems streamline the assessment process.

AI's role in transportation is revolutionary, with self-driving cars poised to transform how we commute. Companies like Tesla and Waymo are at the forefront of developing autonomous vehicles that promise to reduce accidents and improve traffic management.

Challenges and Ethical Considerations

Despite its numerous advantages, AI presents significant challenges and ethical dilemmas. One of the primary concerns is job displacement. As AI systems become more capable, there is a fear that they will replace human workers, leading to widespread unemployment. It is crucial to develop strategies for workforce retraining and transitioning to mitigate this impact.

Privacy is another critical issue. AI systems often require vast amounts of data to function effectively. This data collection can infringe on individual privacy rights, raising concerns about surveillance and data misuse. Implementing robust data protection regulations is essential to address these concerns.

Bias in AI algorithms is also a significant problem. AI systems can perpetuate and even amplify existing biases present in the data they are trained on. This can lead to unfair and discriminatory outcomes. Ensuring diversity in AI development teams and creating mechanisms for bias detection and correction are vital steps in addressing this issue.

The Future of AI

The future of AI holds immense potential. Advances in machine learning, natural language processing, and robotics are expected to drive further innovation. AI could play a crucial role in addressing global challenges such as climate change, healthcare accessibility, and sustainable development.

One of the most exciting prospects is the development of AI that can augment human intelligence. By working alongside humans, AI systems can enhance our cognitive abilities, enabling us to solve complex problems more efficiently. This symbiotic relationship between humans and machines could lead to unprecedented advancements in science, technology, and society.

Conclusion

Artificial Intelligence is undoubtedly revolutionizing the future. Its impact is already evident in numerous fields, and its potential for further transformation is limitless. As we continue to advance AI technologies, it is essential to address the ethical and societal challenges they present. By doing so, we can harness the power of AI to create a better, more equitable future for all.









"""

# Example usage with AI response
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": """You are a detailed and professional assistant. Your task is to generate comprehensive and detailed formatted text. 
                Ensure everything is included !!!!! DO NOT DO NOT DO NOTTTTTTT summarize or truncate any content. OR ELSE I WILL GET REALLY MAD YOU MUST INCLUDE EVERYTHING INCLUDE EVERYTHHIINGNGNGNGNGNGNGGGG!!!!!!!
                Your response must be well-formed and include all details. 

                You must ALWAYS ALWAYS ALWAYS!!!!! output in a structured format as follows AND NO QUOTATION MARKS OR BACKSLASHES!:
                
                {
                    "title": "The Great Renovation",
                    "sections": [
                        {
                            "content": "<specific text content>",
                            "font_name": "<font family>",
                            "font_size": <integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>",
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        },
                        {
                            "content": "<another specific text content>",
                            "font_name": "<another font family>",
                            "font_size": <another integer specifying font size>,
                            "bold": <true or false>,
                            "italic": <true or false>,
                            "alignment": "<right, left, justify or center>",
                            "color": [<integer for red>, <integer for green>, <integer for blue>]
                        }
                        // ... more sections as needed
                    ],
                    "append": <true or false> (this indicates appending to document or not)
                }
                PLEASE! Ensure the output is well-formed and valid."""
        },
        {
            "role": "user",
            "content": "Give whatever file title you want;NO COLONS!ONLY HYPHENS!. Please Include a new title IN THE DOCUMENT, headings if you deem fit, and please format in a nice readable and coherent way.  \
                    Specify the formatting for this text. Please make it a \
                        PROFESSIONALLY formatted text WITH black & blue colors and times new roman font PLEASE. NO NO NO bullet points of ANY KIND unless specified: " + cc_out
        }
    ],
    model="llama3-70b-8192",
)

cc_out2 = chat_completion.choices[0].message.content
title, sections, append = parse_formatted_response(cc_out2)
print(title,'\n',sections,'\n',append)
# Create the Google Doc
document_id = create_google_doc(title)
print(f'Document created with ID: {document_id}')

# Share the document with your personal Google account
user_email = 'sriramnallani35@gmail.com'
share_document_with_user(document_id, user_email)
print(f'Document shared with {user_email}')

# Write content chunks to the Google Doc
write_to_google_doc(document_id, sections, append)

# The URL to access the document
document_url = f'https://docs.google.com/document/d/{document_id}/edit'
print(f'Access the document at: {document_url}')


2024-06-07 16:01:19,498 [MainThread] INFO: file_cache is only supported with oauth2client<4.0.0
2024-06-07 16:01:19,505 [MainThread] INFO: file_cache is only supported with oauth2client<4.0.0
2024-06-07 16:01:19,506 [MainThread] DEBUG: Request options: {'method': 'post', 'url': '/openai/v1/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a detailed and professional assistant. Your task is to generate comprehensive and detailed formatted text. \n                Ensure everything is included !!!!! DO NOT DO NOT DO NOTTTTTTT summarize or truncate any content. OR ELSE I WILL GET REALLY MAD YOU MUST INCLUDE EVERYTHING INCLUDE EVERYTHHIINGNGNGNGNGNGNGGGG!!!!!!!\n                Your response must be well-formed and include all details. \n\n                You must ALWAYS ALWAYS ALWAYS!!!!! output in a structured format as follows AND NO QUOTATION MARKS OR BACKSLASHES!:\n                \n                {\n                    "title": "The G

Artificial-Intelligence-Revolutionizing-the-Future 
 [{'content': 'Artificial Intelligence - Revolutionizing the Future', 'font_name': 'Times New Roman', 'font_size': 24, 'bold': True, 'italic': False, 'alignment': 'center', 'color': (0, 0, 0)}, {'content': 'Introduction', 'font_name': 'Times New Roman', 'font_size': 18, 'bold': True, 'italic': False, 'alignment': 'left', 'color': (0, 0, 0)}, {'content': 'Artificial Intelligence (AI) has emerged as one of the most transformative technologies of the 21st century. Its influence spans across various domains, from healthcare to finance, education to entertainment, and beyond. AI systems are designed to mimic human intelligence, enabling machines to perform tasks that traditionally required human cognitive functions. These tasks include learning, reasoning, problem-solving, perception, and language understanding.', 'font_name': 'Times New Roman', 'font_size': 12, 'bold': False, 'italic': False, 'alignment': 'justify', 'color': (0, 0, 0)}, {

2024-06-07 16:01:29,253 [MainThread] DEBUG: URL being requested: POST https://www.googleapis.com/drive/v3/files/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U/permissions?fields=id&alt=json


Document created with ID: 13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U


2024-06-07 16:01:32,287 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json


Document shared with sriramnallani35@gmail.com


2024-06-07 16:01:33,694 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json
2024-06-07 16:01:34,970 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json
2024-06-07 16:01:36,558 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json
2024-06-07 16:01:37,814 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json
2024-06-07 16:01:39,093 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U:batchUpdate?alt=json
2024-06-07 16:01:40,371 [MainThread] DEBUG: URL being requested: POST https://docs.googleapis.com/v1/documents/13xW

Access the document at: https://docs.google.com/document/d/13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U/edit
